In [1]:
import gradio as gr
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.chat_models import ChatOllama
from langchain.schema import Document
from langchain.memory import ConversationBufferMemory
import os
import json
import datetime

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

if os.path.exists("sii_qa_index"):
    db = FAISS.load_local("sii_qa_index", embedding_model, allow_dangerous_deserialization=True)
else:
    with open('faq_data.json', 'r', encoding='utf-8') as f:
        qa_pairs = json.load(f)

    documents = [Document(page_content=f"Q: {q}\nA: {a}") for q, a in qa_pairs.items()]
    db = FAISS.from_documents(documents, embedding_model)
    db.save_local("sii_qa_index")

retriever = db.as_retriever()


llm = ChatOllama(model="mistral_prompt",temperature=1)


/opt/anaconda3/envs/chbrai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/dw/lrwthtn92hbcrspt70mlprgm0000gn/T/ipykernel_96965/2285697829.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/var/folders/dw/lrwthtn92hbcrspt70mlprgm0000gn/T/ipykernel_96965/2285697829.py:27: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will 

In [2]:
memory=ConversationBufferMemory()
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False,
    memory=memory
)

/var/folders/dw/lrwthtn92hbcrspt70mlprgm0000gn/T/ipykernel_96965/2391637312.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()


In [3]:
rag_chain.invoke({"query": "Hi I am Vasu"})#result = 

{'query': 'Hi I am Vasu',
 'history': '',
 'result': ' Hello Vasu! It seems you are interacting with the Study In India Assistant. The assistant introduced itself as the "Study In India Assistant" in our previous conversation. If you have any questions about studying in India or need assistance, feel free to ask.😊'}

In [4]:
rag_chain.invoke({"query": "What's my name"})

{'query': "What's my name",
 'history': 'Human: Hi I am Vasu\nAI:  Hello Vasu! It seems you are interacting with the Study In India Assistant. The assistant introduced itself as the "Study In India Assistant" in our previous conversation. If you have any questions about studying in India or need assistance, feel free to ask.😊',
 'result': " My name is Study In India Assistant, I'm here to assist you personally 😊"}

In [5]:
print(rag_chain.memory.buffer)

Human: Hi I am Vasu
AI:  Hello Vasu! It seems you are interacting with the Study In India Assistant. The assistant introduced itself as the "Study In India Assistant" in our previous conversation. If you have any questions about studying in India or need assistance, feel free to ask.😊
Human: What's my name
AI:  My name is Study In India Assistant, I'm here to assist you personally 😊


In [6]:
rag_chain.invoke({"query":"What do I do?"})

{'query': 'What do I do?',
 'history': 'Human: Hi I am Vasu\nAI:  Hello Vasu! It seems you are interacting with the Study In India Assistant. The assistant introduced itself as the "Study In India Assistant" in our previous conversation. If you have any questions about studying in India or need assistance, feel free to ask.😊\nHuman: What\'s my name\nAI:  My name is Study In India Assistant, I\'m here to assist you personally 😊',
 'result': "1. To know when the next counseling round is, you can ask the organization handling the process or check their official website for updates.\n2. Yes, you can update your course preferences at any time before the application deadlines for each round. However, it's best to check with the organization first to ensure this option is still available.\n3. If your application is rejected, you'll get a chance to reapply during the next round of counseling. It is essential to re-check your eligibility and modify your course preferences if necessary.\n4. I do